In [1]:
%matplotlib inline

In [2]:
import numpy as np

import sys
sys.path.append("/home/jasonraiti/Documents/GitHub/USC_REU/Project_Files/Jasons_Functions/")

from skeleton_to_graph_from_array import * 

import matplotlib.pyplot as plt

from open_or_show_image import * 

from erosion_dilation_from_array import * 

import networkx as nx

In [3]:
"""Simple travelling salesman problem between cities."""

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


In [4]:

def create_data_model(listOflist,depot_number):
    """Stores the data for the problem."""
    data = {}
    
    # data['distance_matrix'] = [
    #     [0, 123456789, 123456789, 1018, 1631, 1374, 2408, 213, 2571, 875, 1420, 2145, 1972],
    #     [123456789, 0, 1745, 1524, 831, 1240, 959, 2596, 403, 1589, 1374, 357, 579],
    #     [123456789, 1745, 0, 355, 920, 803, 1737, 851, 1858, 262, 940, 1453, 1260],
    #     [1018, 1524, 355, 0, 700, 862, 1395, 1123, 1584, 466, 1056, 1280, 987],
    #     [1631, 831, 920, 700, 0, 663, 1021, 1769, 949, 796, 879, 586, 371],
    #     [1374, 1240, 803, 862, 663, 0, 1681, 1551, 1765, 547, 225, 887, 999],
    #     [2408, 959, 1737, 1395, 1021, 1681, 0, 2493, 678, 1724, 1891, 1114, 701],
    #     [213, 2596, 851, 1123, 1769, 1551, 2493, 0, 2699, 1038, 1605, 2300, 2099],
    #     [2571, 403, 1858, 1584, 949, 1765, 678, 2699, 0, 1744, 1645, 653, 600],
    #     [875, 1589, 262, 466, 796, 547, 1724, 1038, 1744, 0, 679, 1272, 1162],
    #     [1420, 1374, 940, 1056, 879, 225, 1891, 1605, 1645, 679, 0, 1017, 1200],
    #     [2145, 357, 1453, 1280, 586, 887, 1114, 2300, 653, 1272, 1017, 0, 504],
    #     [1972, 579, 1260, 987, 371, 999, 701, 2099, 600, 1162, 1200, 504, 0],
    # ]  # yapf: disable
    
    data['distance_matrix'] = listOflist
    data['num_vehicles'] = 1
    data['depot'] = depot_number
    return data
    # Objective: 7293 miles
    # Route for vehicle 0:
    #  0 -> 7 -> 2 -> 3 -> 4 -> 12 -> 6 -> 8 -> 1 -> 11 -> 10 -> 5 -> 9 -> 0


def print_solution(manager, routing, solution):
    """Prints solution on console."""
    print('Objective: {} miles'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route distance: {}miles\n'.format(route_distance)


def main(listOflist, depot_number):
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model(listOflist,depot_number)

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Setting first solution heuristic.ter
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.AUTOMATIC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(manager, routing, solution)


In [5]:
# start dialation and erosion 

# path to image
path = r'../traveling_salesman_test/zigzag_full.png'
image = open_image(path)

graph = skeleton_to_graph_from_array(image)

# print(type(graph)) #<class 'networkx.classes.graph.Graph'>

A = nx.adjacency_matrix(graph)

ndarray = A.toarray()
listOflist = ndarray.tolist()
print(A[161])
# print(listOflist[0][-1])
print(len(listOflist))
print(len(listOflist[1]))


found image at :  ../traveling_salesman_test/zigzag_full.png

skeleton converted to graph

  (0, 149)	65.01219330881975
162
162


In [6]:

for i in range(0,len(listOflist)):
    not_all_zeros = any(np.array(listOflist[i]))
    if not_all_zeros != True:
        print("something is wrong at point :", i )
        
    for j in range(0,len(listOflist[i])):
        
        
        if listOflist[i][j] == 0 :
            if i != j:
                listOflist[i][j] = 10**10#BIG ASS NUMBER , but preserve diagonal 0s 

# print(listOflist)ter

In [7]:

if __name__ == '__main__':
    main(listOflist,123)


Objective: 380000002877 miles
Route for vehicle 0:
 123 -> 122 -> 150 -> 156 -> 140 -> 135 -> 161 -> 127 -> 126 -> 118 -> 112 -> 113 -> 95 -> 96 -> 115 -> 109 -> 106 -> 104 -> 120 -> 119 -> 103 -> 114 -> 110 -> 131 -> 146 -> 151 -> 152 -> 148 -> 130 -> 132 -> 128 -> 134 -> 153 -> 143 -> 141 -> 159 -> 158 -> 133 -> 138 -> 155 -> 142 -> 139 -> 157 -> 154 -> 160 -> 147 -> 149 -> 145 -> 144 -> 137 -> 136 -> 125 -> 124 -> 121 -> 129 -> 116 -> 117 -> 111 -> 98 -> 108 -> 90 -> 94 -> 75 -> 63 -> 74 -> 85 -> 97 -> 105 -> 107 -> 82 -> 79 -> 77 -> 100 -> 102 -> 101 -> 99 -> 93 -> 88 -> 62 -> 59 -> 92 -> 87 -> 65 -> 61 -> 91 -> 89 -> 69 -> 73 -> 86 -> 60 -> 58 -> 84 -> 80 -> 66 -> 64 -> 83 -> 81 -> 71 -> 67 -> 78 -> 70 -> 76 -> 72 -> 68 -> 2 -> 13 -> 37 -> 15 -> 12 -> 1 -> 48 -> 46 -> 39 -> 10 -> 20 -> 24 -> 27 -> 6 -> 23 -> 30 -> 32 -> 33 -> 34 -> 35 -> 36 -> 57 -> 45 -> 43 -> 41 -> 40 -> 56 -> 54 -> 31 -> 28 -> 21 -> 7 -> 55 -> 52 -> 29 -> 25 -> 22 -> 8 -> 53 -> 50 -> 26 -> 17 -> 9 -> 51 -> 47 -